In [5]:
import wrds
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
import numpy as np
import datetime as dt
from collections import defaultdict

In [6]:
conn = wrds.Connection()

Loading library list...
Done


DATAFMT='STD' and INDFMT='INDL' and CONSOL='C' and POPSRC='D' to retrieve the standardized (as opposed to re-stated data), consolidated (as opposed to pro-forma) data presented in the industrial format (as opposed to financial services format) for domestic companys (as opposed to international firms), i.e., the U.S. and Canadian firms.

PRCC_C: close market price, CSHO: net number of all common shares

Importing linking table to convert permno to gvkey

In [35]:
ASSETG_query = """
SELECT gvkey, datadate, at
FROM comp.funda
WHERE indfmt='INDL' 
AND datafmt='STD' 
AND popsrc='D' 
AND consol='C' 
AND datadate >= '1961-12-31' 
AND datadate <= '2002-12-31'
"""
June_query = """
SELECT permno, date, shrout
FROM crsp.msf
WHERE date >= '1962-06-30' -- Adjusted start date to align with ASSETG data
AND date <= '2003-06-30' -- Adjusted end date
AND EXTRACT(MONTH FROM date) = 6 AND EXTRACT(DAY FROM date) = 30 -- Filter for June 30th
"""

Link_query = """
SELECT Gvkey, Lpermno, Linkdt, Linkenddt
FROM crsp.ccmxpf_linktable
"""

In [8]:
def calculate_assetg(group):
    # Shift within the group
    group['at_lag3'] = group['at'].shift(3)  
    group['at_lag2'] = group['at'].shift(2)
    group['at_lag1'] = group['at'].shift(1)

    # Calculate ASSETG
    group['ASSETG'] = (group['at_lag1'] - group['at_lag2']) / group['at_lag2']
    group['L2ASSETG'] = (group['at_lag2'] - group['at_lag3']) / group['at_lag3']
    return group

In [48]:
# grab data
ASSETG_data = conn.raw_sql(ASSETG_query)
June_data = conn.raw_sql(June_query)
daily_rets = pd.read_csv('daily_rets.csv')
linking_table = conn.raw_sql(Link_query)

convert closing price to abs, filter to only include june 30th entries, merge to shares outstanding df & calculate market value

In [49]:
daily_rets['prc'] = daily_rets['prc'].abs()
daily_rets['date'] = pd.to_datetime(daily_rets['date'])
daily_rets = daily_rets[daily_rets['date'].dt.month == 6][daily_rets['date'].dt.day == 30]

June_data['date'] = pd.to_datetime(June_data['date'])
merged_data = pd.merge(June_data, daily_rets[['permno', 'date', 'prc']], 
                       on=['permno', 'date'], 
                       how='left')
merged_data = merged_data.dropna()
merged_data['MV'] = merged_data['shrout'] * merged_data['prc']

/tmp/ipykernel_98328/3654502307.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  daily_rets = daily_rets[daily_rets['date'].dt.month == 6][daily_rets['date'].dt.day == 30]


switches date to datetime object, creates year column, 
ensures proper sorting & filters by december month (redundent maybe),
applies shifts and calculates ASSETG, drops na and 0 observations, creates deciles

In [50]:
ASSETG_data['datadate'] = pd.to_datetime(ASSETG_data['datadate'])
ASSETG_data['year'] = ASSETG_data['datadate'].dt.year
ASSETG_data = ASSETG_data.sort_values(['gvkey','datadate'])
ASSETG_data = ASSETG_data[ASSETG_data['datadate'].dt.month==12]
ASSETG_data = ASSETG_data.groupby('gvkey').apply(calculate_assetg)
ASSETG_data = ASSETG_data[
    (~pd.isna(ASSETG_data['at_lag1'])) & (ASSETG_data['at_lag1'] != 0) &
    (~pd.isna(ASSETG_data['at_lag2'])) & (ASSETG_data['at_lag2'] != 0) &
    (~pd.isna(ASSETG_data['at_lag3'])) & (ASSETG_data['at_lag3'] != 0)
]
ASSETG_data['decile'] = pd.qcut(ASSETG_data['ASSETG'], 10, labels=False)

Merging linking table to associate MV dates to ASSETG table (not done)

In [51]:
linking_table = linking_table.dropna(subset=['lpermno'])
linking_table['linkdt'] = pd.to_datetime(linking_table['linkdt'])
linking_table['linkenddt'] = pd.to_datetime(linking_table['linkenddt'])

In [ ]:
ASSETG_data = ASSETG_data.reset_index(drop=True)
ASSETG_data = ASSETG_data.merge(linking_table, on='gvkey')

creates dictionary with this strcuture: gvkey(year(ASSETG:x, decile:x)

In [25]:
data_dict = defaultdict(lambda: defaultdict(dict))
for idx, row in ASSETG_data.iterrows():
    data_dict[row['gvkey']][row['year']] = {'ASSETG': row['ASSETG'], 'L2ASSETG': row['L2ASSETG'], 'decile': row['decile']}

For each unique gvkey: checks that iterated year has valid period, grabs decile of iterated year and for each year surrounding it grabs associated ASSETG, sums it with associated decile and offset year key, counts the amount of times its iterated, calculates averages

In [17]:
def has_complete_data_for_period(data_dict, gvkey, center_year):
    for year_offset in range(-4, 6):  # 10-year window: 4 years back, 5 years forward
        year = center_year + year_offset
        if year not in data_dict[gvkey]:
            return False  # Data for this year is missing or incomplete
    return True  # All years in the period have the necessary data

In [ ]:
#Averaging Logic
sum_data = defaultdict(lambda: defaultdict(int))
count_data = defaultdict(lambda: defaultdict(int))
average_data = defaultdict(lambda: defaultdict(float))

sum_data_L2ASSETG = defaultdict(int)
count_data_L2ASSETG = defaultdict(int)
average_data_L2ASSETG = defaultdict(float)

for gvkey in tqdm(ASSETG_data['gvkey'].unique(), desc='Processing stocks'):
    min_year = min(data_dict[gvkey]) + 4
    max_year = max(data_dict[gvkey]) - 5 
    for center_year in tqdm(range(min_year, max_year + 1), desc=f'Processing years for gvkey {gvkey}'):
        if has_complete_data_for_period(data_dict, gvkey, center_year):
            center_year_decile = data_dict[gvkey][center_year]['decile']
            for year_offset in range(-4, 6):
                year = center_year + year_offset
                if year in data_dict[gvkey]:
                    assetg_value = data_dict[gvkey][year]['ASSETG']

                    # Update sum and count
                    sum_data[center_year_decile][year_offset] += assetg_value
                    count_data[center_year_decile][year_offset] += 1

                    # Calculate and update the average
                    current_sum = sum_data[center_year_decile][year_offset]
                    current_count = count_data[center_year_decile][year_offset]
                    average_data[center_year_decile][year_offset] = current_sum / current_count if current_count != 0 else 0
                    
                    if year_offset == 0:
                        l2assetg_value = data_dict[gvkey][year]['L2ASSETG']
                        sum_data_L2ASSETG[center_year_decile] += l2assetg_value
                        count_data_L2ASSETG[center_year_decile] += 1

        else:
            print(f"Incomplete data for gvkey {gvkey} in center year {center_year}")

for decile in sum_data_L2ASSETG:
    if count_data_L2ASSETG[decile] > 0:
        average_data_L2ASSETG[decile] = sum_data_L2ASSETG[decile] / count_data_L2ASSETG[decile]

Creates ASSETG panel

In [22]:
data_for_df = {}
for year_offset in range(-4, 6):
    row_data = []
    for decile in range(10):
            row_data.append(average_data[decile][year_offset])
    data_for_df[year_offset] = row_data

df = pd.DataFrame(data_for_df, index=range(10)).transpose()
df.columns = [f'Decile {i}' for i in range(10)]
df.index.name = 'Year Offset'
df['9-0 Spread'] = df['Decile 9'] - df['Decile 0']